In [1]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install -qU "langchain[mistralai]"
pip install -U langchain-core langchain-mistralai

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -qU langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -qU langchain-core

Note: you may need to restart the kernel to use updated packages.


In [ ]:
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="pr-clear-boutique-29"
MISTRAL_API_KEY="V4ZZ0MqgB08SJtCEup4MPAohR4pK8htc"

In [ ]:
# langsmith e os e getpass para lidar com as chaves de api
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
# chat model
if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("mistral-small-2503", model_provider="mistralai")

In [ ]:
# embedding model
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
# vetor in-memory
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [1]:
# Loading documents em pdf
%pip install -qU langchain_community pypdf pillow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Loading documents em pdf
from langchain_community.document_loaders import PyPDFDirectoryLoader

directory_path = (
    "../../docs/integrations/document_loaders/example_data/layout-parser-paper.pdf"
)
loader = PyPDFDirectoryLoader("example_data/")

docs = loader.load()
docs[0]

print(docs[0].metadata)

In [ ]:
# Splitting documents (testar pra ver se o msm do exemplo de rag funciona pro pyPDFDirectoryLoader)
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split pdf into {len(all_splits)} sub-documents.")

In [ ]:
# Storing documents
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])